In [3]:
import os
from flask import Flask, request, jsonify, render_template, redirect

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import backend as k

Using TensorFlow backend.


In [21]:
def corpus():
    import keras
    # Initiate Paths JK and updated Trump to TrumpSmall
    filepath = ''
    modelname = ''
    
    #define option here
    option = 'option1'

    if option == 'option1':
        filepath = os.path.join('../../../data','text', 'seuss.txt')
        modelname = os.path.join('../../../data', 'weights', 'checkpoint-22-0.8293-seuss.hdf5')
    if option == 'option2':
        filepath = os.path.join('data','text', 'trump.txt')
        modelname = os.path.join('data', 'weights', 'checkpoint-10-1.6465-trump.hdf5')
    if option == 'option3':
        filepath = os.path.join('data','text', 'illiad.txt')
        modelname = os.path.join('data', 'weights', 'checkpoint-19-1.3924-illiad.hdf5')
    if option == 'option4':
        filepath = os.path.join('data','text', 'timemachine.txt')
        modelname = os.path.join('data', 'weights', 'checkpoint-49-1.4686-timemachine.hdf5')

    k.clear_session()

    # read the text file and covert to lowercase
    raw_text = open(filepath, encoding = "ISO-8859-1").read()
    raw_text = raw_text.lower()

    # create mapping of unique chars to integers, and a reverse mapping
    chars = sorted(list(set(raw_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    # summarize the loaded data
    n_chars = len(raw_text)
    n_vocab = len(chars)

    # prepare the dataset of input to output pairs encoded as integers
    seq_length = 100
    dataX = []
    dataY = []
    for i in range(0, n_chars - seq_length, 1):
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)

    # reshape X to be [samples, time steps, features]
    X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
    # normalize
    X = X / float(n_vocab)
    # one hot encode the output variable
    y = np_utils.to_categorical(dataY)

    # define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))

    # load the network weights
    filename = modelname
    model.load_weights(filename)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    seed_char = []

    # pick a random seed
    seed = []
    start = numpy.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    for value in pattern:
        seed_char.append(int_to_char[value])

    if seed:
        seed[0] = ''.join(seed_char)
    if not seed:
        seed.append(''.join(seed_char))

    pred_char = []

    for i in range(280):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        bob = prediction[0]
        index = numpy.random.choice(len(bob), p=bob)
        # index = numpy.argmax(prediction)
        result = int_to_char[index]
        pred_char.append(result)
        # sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print ("\nDone.")

    if predicted_text:
        predicted_text[0] = ''.join(pred_char)
    if not predicted_text:
        predicted_text.append(''.join(pred_char))
    return result

In [22]:
corpus()


Done.


NameError: name 'predicted_text' is not defined